## [Configuration](#Setup)
## [Unit Tests](#Tests)
## [Inputs](#Assumptions)
## Results

# Setup

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go

from datetime import datetime

In [6]:
# set the file name. This needs to be done manually, as Jupyter makes it oddly difficult to query the current notebook filename.
__file__ = 'comparer.ipynb'

import ipytest.magics
import pytest

In [7]:
%%HTML
<style>
    .container { width:100% !important; } 
</style>

In [8]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [9]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

## Assumptions

In [10]:
cpi = 0.028
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_litres_per_100km = 7.0

purchase_price = 35000
age_at_purchase = 0
years_to_model = 15

## Standing Costs

In [11]:
insurance_per_year = 500
registration_per_year = 500
service_cost = 400
service_interval_km = 10000
service_interval_years = 1.0
roadside_assist_per_year = 200

## Depreciation

In [12]:
def depreciation_rate_for_age(age_in_years):
    '''Go with the simple assumption of 15% for first 3 years, and 10% after that.
       This might be too aggressive, but can fine-tune later.
    '''
    if age_in_years < 3:
        return 0.15
    else:
        return 0.10

In [31]:
def calc_depreciation(
    initial_value,
    years=10,
    dep_rate_func=depreciation_rate_for_age):
    """ Calculates the yearly depreciated value for a range of years, 
        given an initial value, the number of years, 
        and a function that gives the depreciation rate for a given year.
        
        Args:
            initial_value (double): the starting value.
            years (int): the number of years to calculate.
            dep_rate_func (callable): Function accepting a single int that 
                returns the depreciation rate to use for the given age in years.
            
        Returns:
            dep_value: numpy array containing the depreciated value for each year.
            yearly_loss: numpy array containing the depreciation loss for 
                each year, defined as the difference in value between the 
                start and end of the year.
    """
    
    dep_value = np.zeros(years)
    yearly_loss = np.zeros(years)
    dep_value[0] = initial_value
    previous = dep_value[0]
    for yr in range(1, years):
        previous *= (1.0 - dep_rate_func(age_at_purchase + yr))
        dep_value[yr] = previous
        yearly_loss[yr-1] = dep_value[yr-1] - dep_value[yr]
        
    # `yearly_depreciation_cost` is the depreciation loss for the given year, 
    # defined as the difference in value between the start and end of the year.
    # To calculate the final year loss, we therefore need to calculate the 
    # depreciated value one year past the range defined by `years_to_model`
    yearly_loss[-1] = dep_value[-1] * dep_rate_func(age_at_purchase + years)
        
    return dep_value, yearly_loss

### Calculate Depreciation

In [32]:
dep_value, yearly_depreciation_cost = calc_depreciation(purchase_price, years_to_model, depreciation_rate_for_age)

In [33]:
line_chart(dep_value)

In [27]:
line_chart(yearly_depreciation_cost)

# Experimental Code

# Compound interest formula
Solves $A = P(1+\frac{r}{n})^{nt}$ for
* P = principal
* r = annual rate
* n = number of times interest is compounded per year
* t = number of years to compound

Returns an array giving the value for each year in the sequence.

In [ ]:
def compound_interest(
    principal,
    annual_rate,
    years,
    compounds_per_year=1,
):
    return np.array([principal * ((1 + annual_rate/compounds_per_year)**(yr*compounds_per_year)) for yr in range(years)])

In [ ]:
indexed_fuel_cost = compound_interest(fuel_cost_per_litre, cpi, years_to_model)
                                      #np.array([fuel_cost_per_litre * (1 + cpi)**yr for yr in range(years_to_model)])
indexed_fuel_cost

In [ ]:
assert len(indexed_fuel_cost) == years_to_model
assert len(indexed_fuel_cost) == len(dep_value)
assert len(indexed_fuel_cost) == len(yearly_depreciation_cost)

# Tests

In [45]:
%%run_pytest -v --tb=line

def flat_rate(a):
    return 0.1

def test_depreciated_value_result_sizes():
    iv = 100
    y = 5
    dv, loss = calc_depreciation(iv, y, flat_rate)
    
    assert len(dv) == y
    assert len(loss) == y
    
def test_depreciated_value_first_year_is_initial_value():
    iv = 100
    y = 5
    dv, loss = calc_depreciation(iv, y, flat_rate)
    
    assert dv[0] == iv
    
def test_depreciated_value():
    iv = 100
    y = 5
    dv, loss = calc_depreciation(iv, y, flat_rate)
    
    assert dv[0] == iv
    assert dv[1] == dv[0] * 0.9
    assert dv[2] == dv[1] * 0.9
    assert dv[3] == dv[2] * 0.9
    assert dv[4] == dv[3] * 0.9
    
def test_loss():
    iv = 100
    y = 5
    dv, loss = calc_depreciation(iv, y, flat_rate)
    
    assert loss[0] == flat_rate(0) * iv
    assert loss[0] == dv[0] * flat_rate(0)
    assert loss[1] == dv[1] - dv[2]
    assert loss[2] == dv[2] - dv[3]
    assert loss[3] == dv[3] * flat_rate(3)
    assert loss[4] == dv[4] * flat_rate(4)
    
    

============================================================ test session starts =============================================================
platform linux -- Python 3.6.5, pytest-3.7.3, py-1.5.4, pluggy-0.7.1 -- /home/daniel/.virtualenvs/ccc/bin/python3
cachedir: .pytest_cache
rootdir: /home/daniel/code/car-cost-calculator, inifile:
collecting ... collected 4 items

comparer.py::test_depreciated_value_result_sizes <- <ipython-input-45-812e2931103c> PASSED                                             [ 25%]
comparer.py::test_depreciated_value_first_year_is_initial_value <- <ipython-input-45-812e2931103c> PASSED                              [ 50%]
comparer.py::test_depreciated_value <- <ipython-input-45-812e2931103c> PASSED                                                          [ 75%]
comparer.py::test_loss <- <ipython-input-45-812e2931103c> FAILED                                                                       [100%]

===========================================================